In [ ]:
using AlfvenDetectors
using PyPlot
using Flux
using BSON

In [ ]:
datapath = "/home/vit/vyzkum/alfven/cdb_data/data_sample/"

In [ ]:
shots = readdir(datapath)

In [ ]:
shotno = 10893
filename = joinpath(datapath, "Jldata$(shotno).h5")

In [ ]:
data = AlfvenDetectors.Float.(AlfvenDetectors.get_ft_signal(filename, AlfvenDetectors.readnormlogupsd));

In [ ]:
pcolormesh(data)

In [ ]:
# now split the data into patches
function split_horizontal(data::AbstractArray,width)
    splits = collect(1:width:size(data,2))
    return map(x->data[:,x[1]:x[2]-1], zip(splits[1:end-1], splits[2:end]))
end
function split_horizontal(data::AbstractVector,width)
    splits = collect(1:width:size(data,2))
    return map(x->data[x[1]:x[2]-1], zip(splits[1:end-1], splits[2:end]))
end
"""
    reshape4conv(data::AbstractVector)

Reshape a vector of array into a M x N x 1 x L array.
"""
function reshape4conv(data::AbstractVector)
    res = Array{typeof(data[1][1]),4}(undef,size(data[1])...,1,length(data))
    for i in 1:length(data_split)
        res[:,:,1,i] = data_split[i]
    end
    return res
end


In [ ]:
width = 32
data_split = split_horizontal(data,width);
# make it a MxNx1xK array
data_array = reshape4conv(data_split);

In [ ]:
for i in 1:30
    figure()
    pcolormesh(data_array[:,:,1,i])
end

In [ ]:
layer = Flux.Conv((3,3), 1=>16, pad=(1,1),relu)
trainx = reshape(data_array[:,:,1,1], size(data_array[:,:,1,1])...,1,1)
trainx512 = trainx[1:end-1,:,:,:]
layer(trainx)

In [ ]:
function update(model, optimiser)
    for p in params(model)
        Δ = Flux.Optimise.apply!(optimiser, p.data, p.grad)
        p.data .-= Δ
        Δ .= 0
    end
end

### Since ConvTranspose does not work, lets try uspcaling

In [ ]:
function honeszeros(T,segment,length,i)
    res = zeros(T,1,segment*length)
    res[((i-1)*segment+1):i*segment] = ones(T,1,segment)
    return res
end
function voneszeros(T,segment,length,i)
    res = zeros(T,segment*length)
    res[((i-1)*segment+1):i*segment] = ones(T,segment)
    return res
end

In [ ]:
@time honeszeros(Float32,2,3,3)
@time voneszeros(Float32,2,3,3)

In [ ]:
hscalemat(T,scale,n) = vcat([honeszeros(T,scale,n,i) for i in 1:n]...)
vscalemat(T,scale,n) = hcat([voneszeros(T,scale,n,i) for i in 1:n]...)

In [ ]:
@time hscalemat(Float32, 2, 3)
@time vscalemat(Float32, 2, 2)

In [ ]:
x = randn(Float32,2,3)
vscalemat(Float32,4,2)*x*hscalemat(Float32,2,3)

In [ ]:
"""
    upscale(x::AbstractArray, scale)

Upscale a 2D array by the integer number given in scale tuple. Works
even for 3D and 4D array in the first two dimensions.
"""
function upscale(x::AbstractArray{T,2}, scale) where T
    m,n = size(x)
    V = vscalemat(T,scale[1],m)
    H = hscalemat(T,scale[2],n)
    return V*x*H
end
function upscale(x::AbstractArray{T,3}, scale) where T
    M,N,C = size(x)
    # this is important - the array must be of the same type as x, not T
    res = Array{typeof(x[1]),3}(undef,M*scale[1],N*scale[2],C)
    for c in 1:C
        res[:,:,c] = upscale(x[:,:,c],scale)
    end
    return Tracker.collect(res)
end
function upscale(x::AbstractArray{T,4}, scale) where T
    M,N,C,K = size(x)
    # this is important - the array must be of the same type as x, not T
    res = Array{typeof(x[1]),4}(undef,M*scale[1],N*scale[2],C,K)
    for c in 1:C
        for k in 1:K
            res[:,:,c,k] = upscale(x[:,:,c,k],scale)
        end
    end
    return Tracker.collect(res)
end

In [ ]:
layer = Flux.Conv((1,1),1=>1)
a = randn(3,2,1,1)
b = layer(a)

In [ ]:
@time upscale(b[:,:,1,1],(2,3))

In [ ]:
@time upscale(b[:,:,:,1],(2,3))

In [ ]:
@time upscale(b,(2,3))

### backprop through upscale

In [ ]:
X = randn(Float32,24,24,1,1);

In [ ]:
model = Flux.Chain(
    # 24x24x2x1
    Flux.Conv((3,3), 1=>4, pad=(1,1)),
    # 24x24x4x1
    x->maxpool(x,(3,2)),
    # 8x12x4x1
    Flux.Conv((3,3), 4=>8, pad=(1,1)),
    # 8x12x8x1
    x->maxpool(x,(2,2)),
    # 4x6x8x1
    x->reshape(x,:,size(x,4)),
    # 192x1
    Flux.Dense(192,64,relu),
    # 64x1
    Flux.Dense(64,192),
    # 192x1
    x->reshape(x,4,6,8,:),
    # 4x6x8x1
    x->upscale(x,(3,2)),
    # 12x12x8x1
    Flux.Conv((3,3), 8=>4, pad=(1,1)),
    # 12x12x4x1
    x->upscale(x,(2,2)),
    # 24x24x4x1
    Flux.Conv((3,3), 4=>1, pad=(1,1))
    # 24x24x1x1
)

In [ ]:
Y=model(X)

In [ ]:
X

In [ ]:
loss(x) = Flux.mse(model(x),x)
opt = ADAM()

In [ ]:
@time L = loss(X)

In [ ]:
@time Flux.back!(L)
@time update(model, opt)

In [ ]:
@time for i in 1:5000
    L=loss(X)
    Flux.back!(L)
    update(model,opt)
    if i%500==0
        println(L)
    end
end

In [ ]:
opt.eta=0.0001

### does backpropagation through cat work? - yes

In [ ]:
X = randn(3,4)
layer = Flux.Dense(3,2)
f(x) = cat(x,zeros(1,size(x,2)),dims=1)
catloss(x) = Flux.mse(f(layer(x)),x)
opt = ADAM()
f(layer(X))

In [ ]:
l=catloss(X)

In [ ]:
Flux.back!(l)
update(layer,opt)

### try the double cat backprop

In [ ]:
function hpad(x::AbstractArray,width)
    M,N,C,K = size(x)
    return cat(x, zeros(typeof(x[1]), M,width,C,K),dims=2)
end
function vpad(x::AbstractArray,width)
    M,N,C,K = size(x)
    return cat(x, zeros(typeof(x[1]), width,N,C,K),dims=1)
end

In [ ]:
function pad(x::AbstractArray{T,4},widths) where T
    M,N,C,K = size(x)
    return cat(cat(x, zeros(T, M,widths[2],C,K),dims=2), zeros(T, widths[1], N+widths[2],C,K),dims=1)
end

In [ ]:
function pad(x::AbstractArray{T,2},widths) where T
    M,N = size(x)
    return cat(cat(x, zeros(T,M,widths[2]),dims=2), zeros(T, widths[1], N+widths[2]),dims=1)
end

In [ ]:
X = randn(Float32,3,4,1,1)
layer = Flux.Conv((2,2),1=>1)
pad(layer(X),(1,1))
doublecatloss(x) = Flux.mse(x,pad(layer(x),(1,1)))
opt = ADAM()

In [ ]:
l = doublecatloss(X)

In [ ]:
Flux.back!(l)
update(layer,opt)

### backpro trough a single Conv layer works

In [ ]:
layer = Flux.Conv((3,3), 1=>1, pad=(1,1))
X = randn(Float32,4,4,1,1)

In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(X)

In [ ]:
Flux.back!(L)
update(layer,opt)

### backprop through conv + maxpool - OK

In [ ]:
layer = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(3,3)),
    x->maxpool(x,(2,2))
    )
X = randn(Float32,4,4,1,1)

In [ ]:
layer(X)

In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(X)

In [ ]:
Flux.back!(L)
update(layer,opt)

### backprop through conv + convtranspose - OK

In [ ]:
layer = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(0,0)),
    Flux.ConvTranspose((3,3), 1=>1)
    )
X = randn(Float32,4,4,1,1)

In [ ]:
layer(X)

In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(X)

In [ ]:
Flux.back!(L)
update(layer,opt)

### backprop through conv + maxpool + convtranspose - OK

In [ ]:
layer = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(0,0)),
    x->maxpool(x,(1,1)),
    Flux.ConvTranspose((3,3), 1=>1)
    )
X = randn(Float32,4,4,1,1)

In [ ]:
layer(X)

In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(X)

In [ ]:
Flux.back!(L)
update(layer,opt)

### backprop through conv + maxpool something + convtranspose - OK

In [ ]:
layer = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(0,0)),
    x->maxpool(x,(2,2)),
    Flux.ConvTranspose((2,2), 1=>1, pad=(-1,-1))
    )
X = randn(Float32,4,4,1,1)

In [ ]:
layer(X)

In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(X)

In [ ]:
Flux.back!(L)
update(layer,opt)

### now put it all together with some dense layersin between

In [ ]:
# this model works well
model = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(0,0)),
    x->maxpool(x,(2,2)),
    Flux.ConvTranspose((2,2), 1=>1, stride=(2,2), pad=(-1,-1))
)

In [ ]:
# this one too
model = Flux.Chain(
    Flux.Conv((3,3), 1=>1, pad=(0,0)),
    x->maxpool(x,(2,2)),
    Flux.ConvTranspose((3,3), 1=>1, stride=(2,2), pad=(0,0)),
    x->pad(x, (1,1))
)

In [ ]:
# this one too
model = Flux.Chain(
    Flux.Conv((3,3), 1=>2, pad=(0,0)),
    x->maxpool(x,(2,2)),
    Flux.ConvTranspose((3,3), 2=>1, stride=(2,2), pad=(0,0)),
    x->pad(x, (1,1))
)

In [ ]:
# 
model = Flux.Chain(
    Flux.Conv((3,3), 1=>2, pad=(0,0)),
    x->maxpool(x,(2,2)),
    x->reshape(x,:,size(x,4)),
    x->reshape(x,)
    Flux.ConvTranspose((3,3), 2=>1, stride=(2,2), pad=(0,0)),
    x->pad(x, (1,1))
)

In [ ]:
# 
model = Flux.Chain(
    Flux.Conv((3,3), 1=>2, pad=(0,0)),
    x->maxpool(x,(2,2),pad=(1,1)),
    Flux.ConvTranspose((3,3), 2=>1, stride=(2,2), pad=(1,1))
)

In [ ]:
model(trainx)

In [ ]:
Flux.Chain(
    Flux.Conv((3,3), 1=>2, pad=(0,0)),
    x->maxpool(x,(2,2),pad=(1,1)))(trainx)


In [ ]:
testloss(x) = Flux.mse(x,layer(x))
opt = ADAM()

In [ ]:
L= testloss(trainx512)

In [ ]:
Flux.back!(L)
update(layer,opt)

In [ ]:
y=reshape(trainx,:,size(trainx,4))
reshape(y,513,32,1,1)

### Lets try autoencoders

In [ ]:
encoder = Flux.Chain(
    Flux.Conv((3,3), 1=>8, pad=(1,1),relu),
    x->maxpool(x,(4,1)),
    
    Flux.Conv((3,3), 8=>16, pad=(1,1), relu),
    x->maxpool(x,(4,2)),
    
    Flux.Conv((3,3), 16=>32, pad=(1,1), relu),
    x->maxpool(x,(4,2)),
    
    x->reshape(x,:,size(x,4)),
    
    Flux.Dense(2048, 16)
)

In [ ]:
y = encoder(trainx);
#_y = y[1:Int(size(y,1)/2),:];
size(y)

In [ ]:
decoder = Flux.Chain(
    Flux.Dense(16,2048,relu),
    x-> reshape(x,8,8,32,size(x,2)),
    
    Flux.ConvTranspose((3,3), 32=>16, stride=(4,2),pad=(0,1)),
    x->pad(x,(1,1)),
    Flux.ConvTranspose((3,3), 16=>8, stride=(4,2),pad=(0,1)),
    x->pad(x,(1,1)),
    Flux.ConvTranspose((3,3), 8=>1, stride=(4,1),pad=(0,1)),
    x->pad(x,(1,0)),
    Flux.ConvTranspose((3,3), 1=>1, stride=(1,1),pad=(1,1))
)

In [ ]:
loss(x) = Flux.mse(x,model(x))
opt = ADAM()

In [ ]:
L = loss(trainx2)

In [ ]:
Flux.back!(L)